In [1]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(ggplot2)
library(readr)
library(gridExtra)
library(dplyr)

ig_30 <- read.csv("/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/sorted_by_ig1030_time_course_30.csv", sep="\t")
ig_origin <- read.csv("/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/output/sorted_by_ig1030_time_course.csv", sep="\t")

In [ ]:
%%R
library(data.table)

ig_origin_dt <- as.data.table(ig_origin)
ig_30_dt <- as.data.table(ig_30)

ig_origin_dt <- ig_origin_dt[order(-ig_1), .SD[1], by = start][, index1 := frank(-ig_1, ties.method = "first")]
ig_30_dt <- ig_30_dt[order(-ig_1), .SD[1], by = start][, index2 := frank(-ig_1, ties.method = "first")]

table_dt <- ig_origin_dt[, .(start, index1)]
table_dt <- merge(table_dt, ig_30_dt[, .(start, index2)], by = "start", all.x = TRUE)

print(head(table_dt))


Key: <start>
           start index1 index2
          <char>  <int>  <int>
1: R-MMU-1006143    939   4565
2: R-MMU-1011576   3632   2705
3: R-MMU-1013012   2548    514
4: R-MMU-1013013   2247    409
5: R-MMU-1022127   3344   1077
6: R-MMU-1028788   3986   2273


In [ ]:
%%R
columns_to_compare <- c("index2")
file_names <- c("ig_30")

pearson_correlations <- list()

for (i in seq_along(columns_to_compare)) {
  result <- cor.test(table_dt$index1, table_dt[[columns_to_compare[i]]], method = "pearson")
  
  pearson_correlations[[i]] <- list(
    comparison = paste("ig_origin_vs", file_names[i], sep = "_"),
    rho = result$estimate,
    p_value = result$p.value,
    t_value = result$statistic
  )
}

print("------pearson_correlations-------")
for (res in pearson_correlations) {
  print(paste0(res$comparison, ": rho = ", round(res$rho, 4), ", p-value = ", format(res$p_value, scientific = TRUE), ", t-value = ", res$t_value))
}


[1] "------pearson_correlations-------"
[1] "ig_origin_vs_ig_30: rho = 0.2727, p-value = 6.884338e-82, t-value = 19.5449993301774"


In [ ]:
%%R
columns_to_compare <- c("index2")
file_names <- c("ig_30")

spearman_correlations <- list()

for (i in seq_along(columns_to_compare)) {
  result <- cor.test(table_dt$index1, table_dt[[columns_to_compare[i]]], method = "spearman")
  
  spearman_correlations[[i]] <- list(
    comparison = paste("ig_origin_vs", file_names[i], sep = "_"),
    rho = result$estimate,
    p_value = result$p.value,
    S_value = result$statistic
  )
}

print("------spearman_correlations-------")
for (res in spearman_correlations) {
  print(paste0(res$comparison, ": rho = ", round(res$rho, 4), ", p-value = ", format(res$p_value, scientific = TRUE), ", S-value = ", res$S_value))
}


[1] "------spearman_correlations-------"
[1] "ig_origin_vs_ig_30: rho = 0.2727, p-value = 6.884338e-82, S-value = 13057294104"
